In [1]:
%pylab inline
from time import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
gender_raw = sc.textFile("s3n://insight-spark-after-dark/gender.csv.gz").repartition(50)
ratings_raw = sc.textFile("s3n://insight-spark-after-dark/ratings.csv.gz")\
                .persist(StorageLevel.MEMORY_AND_DISK_SER)
ratings_rep_raw = sc.emptyRDD()
for i in range(5):
    ratings_rep_raw = ratings_rep_raw.union(ratings_raw)
    
ratings_rep_raw = ratings_rep_raw.repartition(50)

In [3]:
def parse_gender(row):
    split_row = row.split(",")
        
    return (int(split_row[0]), split_row[1])

def parse_ratings(row):
    split_row = row.split(",")
        
    return (int(split_row[0]), int(split_row[1]), int(split_row[2]))

In [4]:
parsed_gender = gender_raw.map(parse_gender)\
                          .setName("gender")\
                          .cache()
parsed_ratings = ratings_rep_raw.map(parse_ratings).setName("ratings").cache()

In [5]:
# do women rate men higher or the other way around?
from_RDD = parsed_ratings.map(lambda r: (r[0], (r[1], r[2])))
#from_RDD.sample(False, 0.001, 20).collect()

In [6]:
joined_from_RDD = from_RDD.join(parsed_gender)
#joined_from_RDD.sample(False, 0.001, 20).collect()

In [7]:
to_RDD = joined_from_RDD.map(lambda r: (r[1][0][0], (r[1][1], r[1][0][1])))
#to_RDD.sample(False, 0.001, 20).collect()

In [8]:
joined_to_RDD = to_RDD.join(parsed_gender)
#joined_to_RDD.sample(False, 0.001, 20).collect()

In [9]:
from_to_RDD = joined_to_RDD.map(lambda r: ((r[1][0][0], r[1][1]), r[1][0][1]))\
                           .setName("from_to")\
                           .persist(StorageLevel.MEMORY_AND_DISK_SER)
from_to_RDD.sample(False, 0.001, 20).collect()

[((u'M', u'F'), 10),
 ((u'F', u'M'), 5),
 ((u'F', u'M'), 5),
 ((u'M', u'M'), 10),
 ((u'M', u'F'), 1),
 ((u'F', u'F'), 1),
 ((u'M', u'F'), 1),
 ((u'M', u'F'), 10),
 ((u'F', u'M'), 10),
 ((u'F', u'M'), 10),
 ((u'M', u'U'), 5),
 ((u'F', u'M'), 10),
 ((u'F', u'F'), 5),
 ((u'F', u'F'), 6),
 ((u'U', u'U'), 4),
 ((u'M', u'F'), 9),
 ((u'M', u'F'), 8),
 ((u'M', u'F'), 7),
 ((u'M', u'F'), 8),
 ((u'M', u'F'), 7),
 ((u'M', u'F'), 6),
 ((u'M', u'F'), 10),
 ((u'M', u'F'), 5),
 ((u'M', u'F'), 8),
 ((u'M', u'F'), 7),
 ((u'U', u'F'), 9),
 ((u'U', u'F'), 8),
 ((u'F', u'M'), 5),
 ((u'M', u'U'), 2),
 ((u'F', u'U'), 1),
 ((u'F', u'F'), 1),
 ((u'M', u'F'), 10),
 ((u'U', u'M'), 1),
 ((u'M', u'U'), 6),
 ((u'M', u'U'), 3),
 ((u'M', u'U'), 3),
 ((u'U', u'U'), 6),
 ((u'F', u'M'), 6),
 ((u'U', u'M'), 8),
 ((u'U', u'F'), 1),
 ((u'M', u'M'), 9),
 ((u'F', u'M'), 7),
 ((u'M', u'M'), 1),
 ((u'F', u'M'), 5),
 ((u'F', u'U'), 9),
 ((u'F', u'U'), 7),
 ((u'F', u'U'), 6),
 ((u'F', u'U'), 1),
 ((u'F', u'U'), 7),
 ((u'F', u'U

In [10]:
start_time = time()
tot_cnt_RDD = from_to_RDD.aggregateByKey((0,0),
                                         lambda acc, val: (acc[0]+val, acc[1]+1),
                                         lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1]))
avg_RDD = tot_cnt_RDD.mapValues(lambda r: (r[0]/float(r[1]), r[1]))

print avg_RDD.collect()
end_time = time()

[((u'M', u'F'), (5.4839443154591, 16160320)), ((u'M', u'M'), (4.461443365411375, 3413550)), ((u'M', u'U'), (3.886141813233456, 4688420)), ((u'U', u'F'), (5.054918391876257, 2225575)), ((u'U', u'U'), (4.782000655415369, 1830900)), ((u'F', u'U'), (5.3471680554624275, 12303825)), ((u'F', u'M'), (6.922842806613474, 35498440)), ((u'F', u'F'), (5.1351401989401655, 6217950)), ((u'U', u'M'), (6.7049789692109245, 4457750))]


In [11]:
print end_time - start_time

13.0981810093


In [12]:
start_time = time()
tot_cnt_RDD2 = from_to_RDD.groupByKey()\
                          .map(lambda r: (r[0], (sum(list(r[1]))/float(len(r[1])), len(r[1]))))
print tot_cnt_RDD2.collect()
end_time = time()

[((u'M', u'F'), (5.4839443154590999, 16160320)), ((u'M', u'M'), (4.461443365411375, 3413550)), ((u'M', u'U'), (3.886141813233456, 4688420)), ((u'U', u'F'), (5.0549183918762566, 2225575)), ((u'U', u'U'), (4.7820006554153691, 1830900)), ((u'F', u'U'), (5.3471680554624275, 12303825)), ((u'F', u'M'), (6.922842806613474, 35498440)), ((u'F', u'F'), (5.1351401989401655, 6217950)), ((u'U', u'M'), (6.7049789692109245, 4457750))]


In [13]:
print end_time - start_time

18.235273838
